In [5]:
import org.apache.spark.sql.functions._
import org.apache.spark.DaliSpark
import collection.JavaConverters._
import spark.implicits._
import collection.mutable._
import scala.util.control._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset

//val filteredByObjective = data.filter($"objectivetype" === "BRAND_AWARENESS" || $"objectivetype" === "WEBSITE_VISIT").repartition(800, $"accounturn")
case class AccountMemberMostRecentEngagementDate(accounturn: String, memberid: Double, mostRecentEngagementDate: String)
//val advertiser = data.select("accounturn").limit(1).collectAsList.asScala(0)
//case class AccountMember(accounturn: String, memberid: Double)

In [ ]:
//val accountMembers = filteredByObjective.groupBy($"accounturn", $"memberid").agg(first("memberid").alias("first")).drop($"first").as[AccountMember].collectAsList().asScala//.foreach(t => storeAccountMemberData(filteredByObjective, t.accounturn, t.memberid))


In [15]:
//DaliSpark.writeDataFrame(accounts.toDF(), "/jobs/tscprep/test/Chad/accounts", scala.collection.immutable.Map(DaliSpark.OVERWRITE_MODE -> "ALL", DaliSpark.OUTPUT_PARALLELISM -> "1000"))
//val accounts = DaliSpark.createDataFrame("/jobs/tscprep/test/Chad/accounts/*").as[String].collectAsList().asScala
//val accounts = Seq("urn:li:sponsoredAccount:508014636")

In [26]:
/*val metrics = Seq(
"landingpageclicks"
"totalengagements",
"otherengagements",
"likes",
"commentlikes",
"comments",
"shares",
"follows"
)

def getMostRecentEngagementDateForMetric(metric: String): Dataset[AccountMemberMostRecentEngagementDate] = {
    val engagements = filteredByObjective.filter(col(metric) > 0).repartition(200)
    val withMostRecentDateOfEngagement = engagements.groupBy($"accounturn", $"memberid").
    agg(max($"datepartition").alias("mostRecentEngagementDate")).
    as[AccountMemberMostRecentEngagementDate]
    withMostRecentDateOfEngagement
}
//val loop = new Breaks;

//loop.breakable {
for(metric <- metrics){
    val mostRecentEngagementDateForMetric = getMostRecentEngagementDateForMetric(metric)
    val writePath = "/jobs/tscprep/test/Chad/Metrics/" + metric
    DaliSpark.writeDataFrame(mostRecentEngagementDateForMetric.toDF(), writePath, scala.collection.immutable.Map(DaliSpark.OVERWRITE_MODE -> "ALL", DaliSpark.OUTPUT_PARALLELISM -> "10"))    
//    loop.break;
}
//}
*/

In [ ]:
import org.apache.spark.sql.DataFrame
import spark.implicits._

/*case class EmptyRow(accounturn: String, memberid: String, objectivetype: String, summation: Double, D0_1: Integer, D2_3: Integer, D4_7: Integer, D8_30: Integer, 
                                   D31_60: Integer, D61_90: Integer, D91_120: Integer,
                                   D121_150: Integer, D151_180: Integer, D181_210: Integer, 
                                   D211_240: Integer, D241_270: Integer, D271_300: Integer,
                                   D301_330: Integer, D331_360: Integer)
*/

//def storeAccountMemberData(filteredByObjective: DataFrame, account: String, memberNumber: Double): Unit = {

//        val accountId = account.replace("urn:li:sponsoredAccount:","")
//        val filterToAccount = filteredByObjective.filter($"accounturn" === account)

//val metric = "landingpageclicks"
val metrics = Seq(
"landingpageclicks",
"totalengagements",
"otherengagements",
"likes",
"commentlikes",
"comments",
"shares",
"follows"
)
val objectives = Seq(
    "BRAND_AWARENESS",
    "WEBSITE_VISIT"
)

val months = Seq(
Seq("2021-07-01-00", "2021-08-01-00"),
Seq("2021-06-01-00", "2021-07-01-00"),
Seq("2021-05-01-00", "2021-06-01-00"),
Seq("2021-04-01-00", "2021-05-01-00"),
Seq("2021-03-01-00", "2021-04-01-00"),    
Seq("2021-02-01-00", "2021-03-01-00"),
Seq("2021-01-01-00", "2021-02-01-00"),
Seq("2020-12-01-00", "2021-01-01-00"),
Seq("2020-11-01-00", "2020-12-01-00"),
Seq("2020-10-01-00", "2020-11-01-00"),
Seq("2020-09-01-00", "2020-10-01-00"),
Seq("2020-08-01-00", "2020-09-01-00")
)

//val objective = objectives(0)
//val metrics = Seq("comments")
val path = "dalids:///data_derived.ads_reporting_engagements_tracking_metrics_adengagementevent_daily"
for(month <- months){
for(metric <- metrics){
for(objective <- objectives){
    val start = month(0)
    val end = month(1)
    print(start,end)
    val filter = scala.collection.immutable.Map(DaliSpark.FILTER_EXP -> s"datepartition >= '$start' and datepartition < '$end'",DaliSpark.PROJECT_COLS -> s"header.memberid,accounturn,$metric,datepartition,objectivetype") // note I moved this back two years, tons of data.
    val data = DaliSpark.createDataFrame(path, filter)//.withColumn("memberId",$"header.memberid").drop($"header")
    
    val withObjectiveType = data.filter($"objectivetype" === objective).repartition()
// you can count the number of empty entries and add these at the end???
    val withMostRecentDateOfEngagement = DaliSpark.createDataFrame("/jobs/tscprep/test/Chad/Metrics/" + metric).as[AccountMemberMostRecentEngagementDate].
        withColumn("mostRecentEngagementDate", substring($"mostRecentEngagementDate", 1, 10)).repartition($"accounturn") // 800 partitions
    val withMostRecentEngagementDate = withObjectiveType.join(withMostRecentDateOfEngagement, Seq("accounturn","memberid"), "left"). // do not consider persons who never engage?  Maybe we can consider these as active members
    withColumn("mostRecentEngagementDate",when($"mostRecentEngagementDate"isNull, "2021-08-01").otherwise($"mostRecentEngagementDate"))
    val asDate = withMostRecentEngagementDate.withColumn("timestamp", substring($"datepartition", 1, 10)).
    drop($"datepartition")
    val withDateDifference = asDate.withColumn("difference", datediff($"mostRecentEngagementDate", $"timestamp")).
    drop($"mostRecentEngagementDate").
    drop($"timestamp").
    repartition(800, $"accounturn").
    cache
    //val historyCutoff = 365*2 // only grabbing one day worth of data anyways
    //val filterByDateCutoff = withDateDifference.filter($"difference" < historyCutoff && $"difference" > 0)//.cache

//    hdfsLogger.info(s"Count of withDateDifference = ${withDateDifference.count}")
    
    
    val working = withDateDifference
    val withBins = working.
      withColumn("D0_1", when($"difference" <= 1,1).otherwise(0)).
      withColumn("D2_3", when($"difference" >= 2 && $"difference" <= 3,1).otherwise(0)).
      withColumn("D4_7", when($"difference" >= 4 && $"difference" <= 7,1).otherwise(0)).
      withColumn("D8_30", when($"difference" >= 8 && $"difference" <= 30,1).otherwise(0)).
      withColumn("D31_60", when($"difference" >= 31 && $"difference" <= 60,1).otherwise(0)).
      withColumn("D61_90", when($"difference" >= 61 && $"difference" <= 90,1).otherwise(0)).
      withColumn("D91_120", when($"difference" >= 91 && $"difference" <= 120,1).otherwise(0)).
      withColumn("D121_150", when($"difference" >= 121 && $"difference" <= 150,1).otherwise(0)).
      withColumn("D151_180", when($"difference" >= 151 && $"difference" <= 180,1).otherwise(0)).
      withColumn("D181_210", when($"difference" >= 181 && $"difference" <= 210,1).otherwise(0)).
      withColumn("D211_240", when($"difference" >= 211 && $"difference" <= 240,1).otherwise(0)).
      withColumn("D241_270", when($"difference" >= 241 && $"difference" <= 270,1).otherwise(0)).
      withColumn("D271_300", when($"difference" >= 271 && $"difference" <= 300,1).otherwise(0)).
      withColumn("D301_330", when($"difference" >= 301 && $"difference" <= 330,1).otherwise(0)).
      withColumn("D331_360", when($"difference" >= 331 && $"difference" <= 360,1).otherwise(0)).
    drop($"difference").
    repartition(800, $"accounturn").
    cache
    
//    hdfsLogger.info(s"Count of withBins = ${withBins.count}")
    

      val aggregate = withBins.groupBy($"accounturn",$"memberid",$"objectivetype").agg(
        sum(col(metric)).alias("summation"),
        sum($"D0_1").alias("D0_1"),
        sum($"D2_3").alias("D2_3"),
        sum($"D4_7").alias("D4_7"),
        sum($"D8_30").alias("D8_30"),
        sum($"D31_60").alias("D31_60"),
        sum($"D61_90").alias("D61_90"),
        sum($"D91_120").alias("D91_120"),
        sum($"D121_150").alias("D121_150"),
        sum($"D151_180").alias("D151_180"),
        sum($"D181_210").alias("D181_210"),
        sum($"D211_240").alias("D211_240"),
        sum($"D241_270").alias("D241_270"),
        sum($"D271_300").alias("D271_300"),
        sum($"D301_330").alias("D301_330"),
        sum($"D331_360").alias("D331_360")
      ).
    repartition().
    cache

//    hdfsLogger.info(s"Count of aggregate = ${aggregate.count}")
    
    val brand_awareness = aggregate.//.filter($"objectivetype" === "BRAND_AWARENESS").
    drop($"objectivetype").
    withColumn("awareness_summation", when($"summation" > 0, 1).otherwise(0)).
    drop($"summation").
    withColumn("awareness_0_1",$"D0_1").
    drop($"D0_1").
    withColumn("awareness_2_3",$"D2_3").
    drop($"D2_3").
    withColumn("awareness_4_7",$"D4_7").
    drop($"D4_7").
    withColumn("awareness_8_30",$"D8_30").
    drop($"D8_30").
    withColumn("awareness_31_60",$"D31_60").
    drop($"D31_60").
    withColumn("awareness_61_90",$"D61_90").
    drop($"D61_90").
    withColumn("awareness_91_120",$"D91_120").
    drop($"D91_120").
    withColumn("awareness_121_150",$"D121_150").
    drop($"D121_150").
    withColumn("awareness_151_180",$"D151_180").
    drop($"D151_180").
    withColumn("awareness_181_210",$"D181_210").
    drop($"D181_210").
    withColumn("awareness_211_240",$"D211_240").
    drop($"D211_240").
    withColumn("awareness_241_270",$"D241_270").
    drop($"D241_270").
    withColumn("awareness_271_300",$"D271_300").
    drop($"D271_300").
    withColumn("awareness_301_330",$"D301_330").
    drop($"D301_330").
    withColumn("awareness_331_360",$"D331_360").
    drop($"D331_360").
    cache
    
//    hdfsLogger.info(s"Count of brand_awareness = ${brand_awareness.count}")
    
      //val joined = brand_awareness.join(website_visit, Seq("accounturn","memberid"), "inner").repartition(800)//.
      val writePath = "/jobs/tscprep/test/Chad/" + metric + "/" + objective + "/" + start
      DaliSpark.writeDataFrame(brand_awareness, writePath, scala.collection.immutable.Map(DaliSpark.OVERWRITE_MODE -> "ALL", DaliSpark.OUTPUT_PARALLELISM -> "10"))
    }
}
}


In [21]:
data.rdd.getNumPartitions // data has a massive number of partitions to work from.

In [28]:
import org.apache.spark.DaliSpark
val result = DaliSpark.createDataFrame("/jobs/tscprep/test/Chad/shares/*")
result.select(sum("visit_summation")).show(false)

In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.DaliSpark
import collection.JavaConverters._
import spark.implicits._
import collection.mutable._
import scala.util.control._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset

// goal is to combine the account/member data for all months and store these elsewhere.
// I also need to combine brand_awareness and website_visit counts into the same dataset
// the default is awareness

// step 1 is to combine the different months
// step 2 is to combine brand awareness and website visits

def aggregateMonths(df: DataFrame): DataFrame = {
          df.groupBy($"accounturn",$"memberid").agg(
        sum("awareness_summation").alias("awareness_summation"),
        sum($"awareness_0_1").alias("awareness_0_1"),
        sum($"awareness_2_3").alias("awareness_2_3"),
        sum($"awareness_4_7").alias("awareness_4_7"),
        sum($"awareness_8_30").alias("awareness_8_30"),
        sum($"awareness_31_60").alias("awareness_31_60"),
        sum($"awareness_61_90").alias("awareness_61_90"),
        sum($"awareness_91_120").alias("awareness_91_120"),
        sum($"awareness_121_150").alias("awareness_121_150"),
        sum($"awareness_151_180").alias("awareness_151_180"),
        sum($"awareness_181_210").alias("awareness_181_210"),
        sum($"awareness_211_240").alias("awareness_211_240"),
        sum($"awareness_241_270").alias("awareness_241_270"),
        sum($"awareness_271_300").alias("awareness_271_300"),
        sum($"awareness_301_330").alias("awareness_301_330"),
        sum($"awareness_331_360").alias("awareness_331_360")
      ).
    repartition().
    cache
}

val metrics = Seq(
"landingpageclicks",
"totalengagements",
"otherengagements",
"likes",
"commentlikes",
"comments",
"shares",
"follows"
)
//val metric = "landingpageclicks"
for(metric <- metrics){
    val basePath = "/jobs/tscprep/test/Chad/" + metric
    val websiteVisits = DaliSpark.createDataFrame(basePath + "/" + "WEBSITE_VISIT" + "/*")

    val websiteVisitsAggregate = aggregateMonths(websiteVisits).
        withColumnRenamed("awareness_summation", "visit_summation").
        withColumnRenamed("awareness_0_1","visit_0_1").
        withColumnRenamed("awareness_2_3","visit_2_3").
        withColumnRenamed("awareness_4_7","visit_4_7").
        withColumnRenamed("awareness_8_30","visit_8_30").
        withColumnRenamed("awareness_31_60","visit_31_60").
        withColumnRenamed("awareness_61_90","visit_61_90").
        withColumnRenamed("awareness_91_120","visit_91_120").
        withColumnRenamed("awareness_121_150","visit_121_150").
        withColumnRenamed("awareness_151_180","visit_151_180").
        withColumnRenamed("awareness_181_210","visit_181_210").
        withColumnRenamed("awareness_211_240","visit_211_240").
        withColumnRenamed("awareness_241_270","visit_241_270").
        withColumnRenamed("awareness_271_300","visit_271_300").
        withColumnRenamed("awareness_301_330","visit_301_330").
        withColumnRenamed("awareness_331_360","visit_331_360")

    val visitWritePath = s"/jobs/tscprep/test/Chad/Combined/$metric/visit"
    DaliSpark.writeDataFrame(websiteVisitsAggregate, visitWritePath, scala.collection.immutable.Map(DaliSpark.OVERWRITE_MODE -> "ALL", DaliSpark.OUTPUT_PARALLELISM -> "50"))

    val brandAwareness = DaliSpark.createDataFrame(basePath + "/" + "BRAND_AWARENESS" + "/*")
    val brandAwarenessAggregate = aggregateMonths(brandAwareness)
    val awarenessWritePath = s"/jobs/tscprep/test/Chad/Combined/$metric/awareness"
    DaliSpark.writeDataFrame(brandAwarenessAggregate, awarenessWritePath, scala.collection.immutable.Map(DaliSpark.OVERWRITE_MODE -> "ALL", DaliSpark.OUTPUT_PARALLELISM -> "50"))
}



In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.DaliSpark
import collection.JavaConverters._
import spark.implicits._
import collection.mutable._
import scala.util.control._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset

val metrics = Seq(
"landingpageclicks",
"totalengagements",
"otherengagements",
"likes",
"commentlikes",
"comments",
"shares",
"follows"
)
for(metric <- metrics){
    val awareness = DaliSpark.createDataFrame(s"/jobs/tscprep/test/Chad/Combined/$metric/awareness").repartition(800)
    val visit = DaliSpark.createDataFrame(s"/jobs/tscprep/test/Chad/Combined/$metric/visit").repartition(800)
    val joined: DataFrame = awareness.join(visit, Seq("accounturn","memberid"), "inner").
    drop($"memberid").
    drop($"accounturn")
    val joinedWritePath = s"/jobs/tscprep/test/Chad/Joined/$metric"
    //if(DaliSpark.createDataFrame(joinedWritePath).rdd.isEmpty){
//        println(s"$metric")
//        DaliSpark.writeDataFrame(joined, joinedWritePath, scala.collection.immutable.Map(DaliSpark.OVERWRITE_MODE -> "ALL", DaliSpark.OUTPUT_PARALLELISM -> "5"))
        joined.
    repartition(20).
    write.mode("overwrite").option("header", "true").option("sep", "|").csv(joinedWritePath)
    //}
}


In [16]:
print(awareness)

In [5]:
import org.apache.spark.sql.functions._
import org.apache.spark.DaliSpark
import collection.JavaConverters._
import spark.implicits._
import collection.mutable._
import scala.util.control._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset

// purpose is to aggregate files
val metrics = Seq(
"landingpageclicks",
"totalengagements",
"otherengagements",
"likes",
"commentlikes",
"comments",
"shares",
"follows"
)
val objectives = Seq(
    "BRAND_AWARENESS",
    "WEBSITE_VISIT"
)

val months = Seq(
Seq("2021-07-01-00", "2021-08-01-00"),
Seq("2021-06-01-00", "2021-07-01-00"),
Seq("2021-05-01-00", "2021-06-01-00"),
Seq("2021-04-01-00", "2021-05-01-00"),
Seq("2021-03-01-00", "2021-04-01-00"),    
Seq("2021-02-01-00", "2021-03-01-00"),
Seq("2021-01-01-00", "2021-02-01-00"),
Seq("2020-12-01-00", "2021-01-01-00"),
Seq("2020-11-01-00", "2020-12-01-00"),
Seq("2020-10-01-00", "2020-11-01-00"),
Seq("2020-09-01-00", "2020-10-01-00"),
Seq("2020-08-01-00", "2020-09-01-00")
)

//val objective = objectives(0)
//val metrics = Seq("comments")
import scala.util.control._

// create a Breaks object as follows
val loop = new Breaks;

// Keep the loop inside breakable as follows
loop.breakable{
for(metric <- metrics){  
    val loadPath = s"/jobs/tscprep/test/Chad/Joined/$metric"
    //val data = DaliSpark.createDataFrame(loadPath)
    val data = spark.read.format("csv").option("header", "true").option("sep","|").load(loadPath)
    val tempSaveLocation = "/jobs/tscprep/test/Chad/temp/" + metric
    DaliSpark.writeDataFrame(data.repartition(3), tempSaveLocation, scala.collection.immutable.Map(DaliSpark.OVERWRITE_MODE -> "ALL", DaliSpark.OUTPUT_PARALLELISM -> "3"))
    //val reload = DaliSpark.createDataFrame(tempSaveLocation)
    val reload = spark.read.format("csv").option("header", "true").option("sep","|").load(tempSaveLocation)
    println(s"${reload.count} and ${data.count}")
    reload.repartition(5).
    drop($"accounturn").
    drop($"memberid").
    write.mode("overwrite").option("header", "true").option("sep", "|").csv(loadPath)
    //DaliSpark.writeDataFrameAsCsv(reload.repartition(1), loadPath, scala.collection.immutable.Map(DaliSpark.OVERWRITE_MODE -> "ALL", DaliSpark.OUTPUT_PARALLELISM -> "1"))
}
}

In [23]:
//spark.read.format("csv").options(header="true").load(loadPath)
